In [1]:
import pandas as pd
from auto_detect import DataAutoDetecter
from masking_funcs import Masker
from data_anonymizer import DataAnonymizer

In [2]:
df = pd.read_csv('adult.csv')

In [3]:
anonymizer = DataAnonymizer(df)

Auto Detecting Data Properties...
Constructing Properties Frame


In [4]:
anonymizer.properties_frame

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continous,Others,Non-Sensitive
workclass,Categorical,Others,Non-Sensitive
fnlwgt,Continous,Others,Non-Sensitive
education,Categorical,Others,Non-Sensitive
educational-num,Continous,Others,Non-Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [5]:
anonymizer.properties_frame

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continous,Others,Non-Sensitive
workclass,Categorical,Others,Non-Sensitive
fnlwgt,Continous,Others,Non-Sensitive
education,Categorical,Others,Non-Sensitive
educational-num,Continous,Others,Non-Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [6]:
anonymizer.change_property('race', 'column_type', 'Rawr')

AttributeError: Unknown column type specified. Please specify one of the approved types in ['Continous', 'Categorical', 'Datetime']